# Python En Parrellèle

Avant de commencer à vous expliquer comment faire du threading et du multiprocessing en python, vous devez avoir une
idée de base de comment fonctionne votre ordinateur. Sans ces connaissances de base, il sera vraiment difficile de
comprendre ce que vous codez.

Un CPU est en ensemble de Core. Un core est un unité de calcul ou “processing unit”. C’est essentiellement une petite
calculatrice qui fait des petites opérations vraiment simples et qui ne peut seulement faire un seul petit calcul à la
fois. Ça veut dire que si votre CPU possède 9 cores comme dans la figure, alors votre CPU peut faire seulement 9
opérations à la fois. Évidemment, vous devez vous doutez que vous arrivez à demander à votre CPU de faire plus que 9
tâches à fois… Sinon comment pourriez vous faire pour être sur discord avec vos en même temps que de jouer à Valheim
sur le serveur où vous êtes host toute en écoutant votre cours sur zoom… Eh bien, c’est l’utilitée des threads. Un
thread est un programme ou un ensemble d’opérations qu’on demande à un core d’exécuter. Afin qu’un core soit en mesure
d’exécuter les opérations nécessaire à plusieurs programmes à la fois, il alterne entre ceux-ci en exécutant une partie
de chaque programme. Il peut donc y avoir plusieurs threads sur un seul core et les programmes sembleront s'exécuter en
même temps. Toutefois, il ne faut pas se leurrer. En ayant deux programmes roulant sur le même core, le temps
d'exécution sera plus grand ou égale au temps que ça aurait prit pour lancer ces deux programmes de façon séquentiel.
Par contre, si on lance ces deux même threads sur deux cores différent, ce qu’on appelle le multiprocessing, les deux
programmes se feront réellement en même temps et donc le temps d’exécution finale sera égale au temps du programme le
plus long.

Dans cette présentation, je vais vous montrer comment lancer plusieurs threads en même temps sur un seul core. Par
après nous allons voir comment lancer des programmes sur plusieurs cores différents afin d’avoir des programmes
s’exécutant réellement en parallèles dans l’objectif de sauver du temps.


![CPU_Shm](figures/CPU_Shm.png)

In [1]:
import time
import functools
import numpy as np
import multiprocessing as mp
import psutil
from functools import partial

In [2]:
def benchmark(_func=None, *, box_length=50):
    def decorator_print_func_section(func):
        box_char = '-'
        title = (box_char*box_length) + ' ' + func.__name__ + ' ' + (box_char*box_length)

        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            print(title)
            start_time = time.time()
            out = func(*args, **kwargs)
            stats_line = f"func: {func.__name__} executed in {time.time() - start_time:.5f}s."
            print(stats_line)
            print(box_char*len(title))
            return out
        return wrapper

    if _func is None:
        return decorator_print_func_section
    else:
        return decorator_print_func_section(_func)

In [3]:
def squared(x):
    return x**2

@benchmark
def array_squared_sp(array):
    return [squared(x) for x in array]

@benchmark
def array_squared_mp(array):
    with mp.Pool(psutil.cpu_count(logical=False)) as pool:
        out = pool.map(squared, array)
    return out

In [1]:
numbers = np.random.uniform(0, 1, size=10)

array_squared_sp(numbers)
array_squared_mp(numbers)

NameError: name 'np' is not defined